In [ ]:
import pandas as pd

# Load Data
df = pd.read_csv('shipment_booking_data_2021_2025.csv')
df['booking_date'] = pd.to_datetime(df['booking_date'])

# --- ANALYSIS 1: LAST 5 DAYS ---
print("\n=== PATTERN FOR LAST 5 DAYS ===")

# Get last 5 unique dates
unique_dates = sorted(df['booking_date'].unique())
last_5_dates = unique_dates[-5:]
latest_date = last_5_dates[-1]

# Filter data
recent_df = df[df['booking_date'].isin(last_5_dates)].copy()

# 1. Volume per Day
daily_vol = recent_df.groupby('booking_date').size()
print("\n[Daily Volume]")
print(daily_vol)

# 2. Distribution of Volume (Shipment Type breakdown per day)
dist = recent_df.groupby(['booking_date', 'shipment_type']).size().unstack(fill_value=0)
# Calculate percentages
dist_pct = dist.div(dist.sum(axis=1), axis=0) * 100

print("\n[Shipment Type Distribution (%)]")
print(dist_pct.round(1))


=== PATTERN FOR LAST 5 DAYS ===

[Daily Volume]
booking_date
2025-12-27     98
2025-12-28    109
2025-12-29    102
2025-12-30    105
2025-12-31    113
dtype: int64

[Shipment Type Distribution (%)]
shipment_type   Air  Express  International  Surface
booking_date                                        
2025-12-27     28.6     20.4            7.1     43.9
2025-12-28     32.1     22.9            5.5     39.4
2025-12-29     31.4     20.6            2.0     46.1
2025-12-30     33.3     22.9            8.6     35.2
2025-12-31     28.3     22.1            7.1     42.5


In [27]:
# --- ANALYSIS 2: HISTORICAL COMPARISON (SAME WINDOW ACROSS YEARS) ---
target_date = latest_date
window = 10 # days before target

print(f"\n=== HISTORICAL COMPARISON (Year-over-Year) ===")
print(f"Target Window: {window} days up to {target_date.month_name()} {target_date.day}")

# Create the window of Month-Days we care about
dates_of_interest = []
for i in range(window + 1): 
    d = target_date - pd.Timedelta(days=i)
    dates_of_interest.append((d.month, d.day))

# Filter entire dataset for these Month-Day combos
df['month_day'] = list(zip(df['booking_date'].dt.month, df['booking_date'].dt.day))
historical_df = df[df['month_day'].isin(dates_of_interest)].copy()
historical_df['year'] = historical_df['booking_date'].dt.year

# Aggregations
# 1. Volume per Year for this window
yearly_vol = historical_df.groupby('year').size()
print(f"\n[Total Volume in Window ({window+1} days) - Year over Year]")
print(yearly_vol)

# 2. Distribution per Year (ABSOLUTE & PERCENTAGE)
yearly_dist = historical_df.groupby(['year', 'shipment_type']).size().unstack(fill_value=0)
yearly_dist_pct = yearly_dist.div(yearly_dist.sum(axis=1), axis=0) * 100

print("\n[Shipment Type ABSOLUTE VOLUMES per Year]")
print(yearly_dist)

print("\n[Shipment Type PERCENTAGE per Year (%)]")
print(yearly_dist_pct.round(1))


=== HISTORICAL COMPARISON (Year-over-Year) ===
Target Window: 10 days up to December 31

[Total Volume in Window (11 days) - Year over Year]
year
2021    1185
2022    1155
2023    1227
2024    1114
2025    1137
dtype: int64

[Shipment Type ABSOLUTE VOLUMES per Year]
shipment_type  Air  Express  International  Surface
year                                               
2021           409      220             64      492
2022           404      241             51      459
2023           425      260             57      485
2024           388      235             63      428
2025           392      236             56      453

[Shipment Type PERCENTAGE per Year (%)]
shipment_type   Air  Express  International  Surface
year                                                
2021           34.5     18.6            5.4     41.5
2022           35.0     20.9            4.4     39.7
2023           34.6     21.2            4.6     39.5
2024           34.8     21.1            5.7     38.4
2025     